In [15]:
from app.pdf_agent import RAGRetriever, Embedder, ChromaStore, CrossEncoderReranker, PDFIngestor, PDFPartitioner, PDFSplitter
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

model = AutoModel.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    dtype=torch.float16,  # 🔑 FP16
    device_map="auto"           # 🔑 moves model to GPU
)

model.eval()
model.to("mps")

store = ChromaStore(path="./chroma_db", collection_name="rag")

embedder = Embedder(tokenizer, model, batch_size=8)

reranker = CrossEncoderReranker(
    model_name="cross-encoder/ms-marco-MiniLM-L-6-v2",
    # model_name="cross-encoder/nli-deberta-v3-base",
    top_k=8
)

retriever = RAGRetriever(embedder, store, reranker)

In [16]:
splitter = PDFSplitter(20)
partitioner = PDFPartitioner()

ingestor = PDFIngestor(splitter, partitioner, embedder, store)


In [ ]:
ingestor.ingest("data/downloads/harry_potter.pdf", "harry_potter", "data/downloads/harry_potter_split", lambda x: x)

In [18]:
import os
questions = [
    "How does Harry first learn that he is a wizard?",
    "Why did Harry go to live with the Dursleys after his parents' death?",
    "What role does Hagrid play in introducing Harry to the wizarding world?",
    "How does Harry get to Diagon Alley for the first time?",
    "What item does Harry buy that later becomes important in the story?",
    "Why does the Sorting Hat consider placing Harry in Slytherin?",
    "What qualities ultimately lead the Sorting Hat to place Harry in Gryffindor?",
    "How do Harry, Ron, and Hermione become friends?",
    "What is unusual about Harry's ability during his first Quidditch match?",
    "Why does Snape appear to dislike Harry from their first meeting?",
    "What is the significance of the three-headed dog guarding the trapdoor?",
    "Why are the characters trying to protect the Philosopher's Stone?",
    "What challenges do Harry, Ron, and Hermione face beneath the trapdoor?",
    "Why does Ron sacrifice himself during the wizard's chess game?",
    "How does Hermione help Harry during the potion riddle?",
    "Why is Professor Quirrell able to pass through the magical protections?",
    "What role does the Mirror of Erised play in the climax of the story?",
    "Why is Harry able to touch the Philosopher's Stone when Quirrell cannot?",
    "How does Dumbledore explain the power that protected Harry from Voldemort?",
    "How does the House Cup outcome reflect the values emphasized in the story?",
]
questions_rewritten = [
    "Harry first learns that he is a wizard when",
    "Harry goes to live with the Dursleys after his parents' death because",
    "Hagrid introduces Harry to the wizarding world by",
    "Harry gets to Diagon Alley for the first time by",
    "The item Harry buys that later becomes important in the story is",
    "The Sorting Hat considers placing Harry in Slytherin because",
    "Harry is ultimately placed in Gryffindor due to",
    "Harry, Ron, and Hermione become friends after",
    "During his first Quidditch match, Harry shows an unusual ability when",
    "Snape appears to dislike Harry from their first meeting because",
    "The significance of the three-headed dog guarding the trapdoor is that",
    "The characters are trying to protect the Philosopher's Stone because",
    "Beneath the trapdoor, Harry, Ron, and Hermione face challenges such as",
    "Ron sacrifices himself during the wizard's chess game in order to",
    "Hermione helps Harry during the potion riddle by",
    "Professor Quirrell is able to pass through the magical protections because",
    "The Mirror of Erised plays a crucial role in the climax by",
    "Harry is able to touch the Philosopher's Stone while Quirrell cannot because",
    "Dumbledore explains that the power protecting Harry comes from",
    "The House Cup outcome reflects the story's values by"
]


In [19]:
results = retriever.search(questions)
ABLATION_PATH = "data/rag_ablations"
RERANK = ABLATION_PATH + "/rag_rerank"

os.makedirs("data/rag_ablations/rag_rerank", exist_ok=True)
for i, result in enumerate(results):
    query_res = [doc for _, doc, _ in result]
    with open(RERANK + f"/document_{i}.txt", "w+") as f:
        f.write(f"Question: {questions[i]} \n")
        f.write("\n".join(query_res))

[['[harry_potter_p0021-0040.pdf | Page 40] “Kept what from me?” said Harry eagerly.\n\n“STOP! I FORBID YOU!” yelled Uncle Vernon in panic.\n\nAunt Petunia gave a gasp of horror.\n\n“Ah, go boil yer heads, both of yeh,” said Hagrid. “Harry — yer a\n\nwizard.”\n\nThere was silence inside the hut. Only the sea and the whistling wind could be heard.\n\n“I’m a what?” gasped Harry.\n\n“A wizard, o’ course,” said Hagrid, sitting back down on the sofa, which groaned and sank even lower, “an’ a thumpin’ good ‘un, I’d say, once yeh’ve been trained up a bit. With a mum an’ dad like yours, what else would yeh be? An’ I reckon it’s abou’ time yeh read yer letter.”\n\nHarry stretched out his hand at last to take the yellowish envelope, addressed in emerald green to Mr. H. Potter, The Floor, Hut-on-the-Rock, The Sea. He pulled out the letter and read:\n\nHOGWARTS SCHOOL of WITCHCRAFT and WIZARDRY\n\nHeadmaster: ALBUS DUMBLEDORE\n\n(Order of Merlin, First Class, Grand Sorc., Chf. Warlock, Supreme Mugw

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.65it/s]


In [20]:
q_emb = retriever.embedder.embed(questions)
# 2️⃣ High-recall vector search
results, metas = retriever.vector_store.search(
    query_embedding=q_emb,
    top_k=retriever.recall_k,
)

NO_RERANK = ABLATION_PATH + "/rag_no_rerank"

os.makedirs("data/rag_ablations/rag_no_rerank", exist_ok=True)
for i, result in enumerate(results):
    query_res = [doc for doc in result[:5]]
    with open(NO_RERANK + f"/document_{i}.txt", "w+") as f:
        f.write(f"Question: {questions[i]} \n")
        f.write("\n".join(query_res))


In [24]:
os.makedirs("data/rag_ablations/final", exist_ok=True)
for i in range(len(questions)):
    doc = "You are an LLM as a judge. You are supposed to judge between the 2 RAG result below. Can you tell me which one is better? Give a score for each 1 to 5 for each retrieval with 5 being good in the format of Retrieval 1: <score>, Retrieval 2: <score> and a short reason why. \n Retrieval 1\n"
    with open(NO_RERANK + f"/document_{i}.txt") as f:
        doc += f.read() + "\n Retrieval 2\n"

    with open(RERANK + f"/document_{i}.txt") as f:
        doc += f.read()

    with open(f"data/rag_ablations/final/document_{i}.txt", "w+") as f:
        f.write(doc)

# Table Ingestion

In [25]:
from typing import Callable
from pathlib import Path
import shelve
import uuid
from app.pdf_agent import PDFTableExtractor
import fitz

store = ChromaStore(path="./chroma_db", collection_name="table")
extractor = PDFTableExtractor(device="mps")

class PDFIngestor:
    def __init__(
        self,
        splitter: PDFSplitter,
        partitioner: PDFPartitioner,
        embedder: Embedder,
        store: ChromaStore,
        table_extractor: PDFTableExtractor
    ):
        self.splitter = splitter
        self.partitioner = partitioner
        self.embedder = embedder
        self.store = store
        self.global_chunk_id = 0
        self.table_extractor = table_extractor
        self.db_path = "data/user_data"

    def ingest(
        self,
        pdf_path: str,
        source_name: str,
        split_dir: str,
        preprocess_text: Callable[[str], str]
    ):
        part_pdfs = self.splitter.split(pdf_path, split_dir)
        print(f"Split into {len(part_pdfs)} PDF parts")
        # Images
        for part_idx, part in enumerate(part_pdfs, start=1):
            part_pdf = part["path"]
            part_start_page = part["start_page"]

            doc = fitz.open(part_pdf)

            for page_idx, page in enumerate(doc, start=1):
                original_page = part_start_page + page_idx - 1
                extracted, _ = self.table_extractor.extract(page)

                for table in extracted:
                    if not table.markdown:
                        continue

                    content = (
                        f"Caption: {table.caption}\n Table: \n{table.markdown}"
                        if table.caption
                        else table.markdown
                    )

                    embedding_text = table.caption or table.markdown
                    embeddings = self.embedder.embed([embedding_text])

                    metadata = {
                        "source": source_name,
                        "part_pdf": Path(part_pdf).name,
                        "page": original_page,
                        "type": "table",
                    }

                    doc_id = f"{source_name}_table_{self.global_chunk_id}"

                    self.store.add(
                        documents=[content],
                        embeddings=embeddings.tolist(),
                        metadatas=[metadata],
                        ids=[doc_id],
                    )

                    self.global_chunk_id += 1
        
        
        # For the text
        for idx, part in enumerate(part_pdfs, start=1):
            part_pdf = part["path"]
            part_start_page = part["start_page"]


            chunks = self.partitioner.partition(part_pdf)
            if not chunks:
                continue

            texts = []
            metadatas = []

            for c in chunks:
                chunk_page = c["page"]
                original_page = (
                    part_start_page + chunk_page - 1
                    if chunk_page is not None
                    else None
                )

                texts.append(
                    f"[{Path(part_pdf).name} | Page {original_page}] {preprocess_text(c['text'])}"
                )

                metadatas.append({
                    "source": source_name,
                    "part_pdf": Path(part_pdf).name,
                    "page": original_page,
                    "type": c["type"],
                })

            ids = [
                f"{source_name}_{self.global_chunk_id + i}"
                for i in range(len(chunks))
            ]

            embeddings = self.embedder.embed(texts)

            self.store.add(
                documents=texts,
                embeddings=embeddings.tolist(),
                metadatas=metadatas,
                ids=ids
            )

            self.global_chunk_id += len(chunks)

            print(f"[Part {idx}/{len(part_pdfs)}] Ingested {len(chunks)} chunks")




Downloaded to data/models/yolov11x_best.pt


/Users/viriyadhika/Documents/5.UofT/Projects/SmallAgent/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/viriyadhika/Documents/5.UofT/Projects/SmallAgent/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/viriyadhika/Documents/5.UofT/Projects/SmallAgent/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWa

In [26]:
store.client.delete_collection("table")
store = ChromaStore(path="./chroma_db", collection_name="table")
ingestor = PDFIngestor(splitter, partitioner, embedder, store, extractor)
retriever = RAGRetriever(embedder, store, reranker)


In [27]:
ingestor.ingest("data/downloads/manual_down/diffusion_policy.pdf", "diffusion_policy", "data/downloads/diffusion_policy_split", lambda x: x)

Split into 1 PDF parts

0: 640x512 3 List-items, 1 Page-header, 1 Picture, 1 Section-header, 8 Texts, 535.4ms
Speed: 6.7ms preprocess, 535.4ms inference, 40.1ms postprocess per image at shape (1, 3, 640, 512)

=============== FIGURE CAPTION SCORING (CLIP ONLY) ===============
Candidate 1
Text:
Fig. 1: Policy Representations. a) Explicit policy with different types of action representations. b) Implicit policy learns an energy function
conditioned on both action and observation and optimizes for actions that minimize the energy landscape c) Diffusion policy refines noise
into actions via a learned gradient field. This formulation provides stable training, allows the learned policy to accurately model multimodal
action distributions, and accommodates high-dimensional action sequences.
CLIP similarity : 0.3609
------------------------------------------------------------
Candidate 2
Text:
1 Columbia University
2 Toyota Research Institute
3 MIT
https://diffusion-policy.cs.columbia.edu
CLIP 

Batches: 100%|██████████| 1/1 [00:00<00:00, 30.26it/s]



=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

| pθ(a|o) | = e−Eθ |
| --- | --- |
|  | Z(o,θ) | 

Rank 1 | Score -0.5502
e−Eθ (o,a) +∑
Nneg
j=1e−Eθ (o,ea j) )
(7)
------------------------------------------------------------
Rank 2 | Score -3.5767
LinfoNCE = −log(
e−Eθ (o,a)
------------------------------------------------------------
Rank 3 | Score -8.0557
where Z(o,θ) is an intractable normalization constant (with
respect to a).
To train the EBM for implicit policy, an InfoNCE-style loss
function is used, which equates to the negative log-likelihood
of Eq 6:
------------------------------------------------------------

tensor([[0.2721, 0.7279]], device='mps:0')

0: 640x512 2 Pictures, 1 Section-header, 2 Tables, 8 Texts, 473.9ms
Speed: 2.0ms preprocess, 473.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 512)

=============== FIGURE CAPTION SCORING (CLIP ONLY) ===============
Candid

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.41it/s]



=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

|  | Lift ph | mh | Can ph | mh | Square ph | mh | Transport ph | mh | ToolHang ph | Push-T ph |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| LSTM-GMM [29] | 1.00/0.96 | 1.00/0.95 | 1.00/0.88 | 0.98/0.90 | 0.82/0.59 | 0.64/0.38 | 0.88/0.62 | 0.44/0.24 | 0.68/0.49 | 0.69/0.54 |
| IBC [12] | 0.94/0.73 | 0.39/0.05 | 0.08/0.01 | 0.00/0.00 | 0.03/0.00 | 0.00/0.00 | 0.00/0.00 | 0.00/0.00 | 0.00/0.00 | 0.75/0.64 |
| DiffusionPolicy-C | 1.00/1.00 | 1.00/1.00 | 1.00/0.97 | 1.00/0.96 | 0.98/0.92 | 0.98/0.84 | 1.00/0.93 | 0.89/0.69 | 0.95/0.73 | 0.91/0.84 | 

Rank 1 | Score -7.7686
TABLE II: Behavior Cloning Benchmark (Visual Policy) Performance are reported in the same format as in Tab I. LSTM-GMM numbers
were reproduced to get a complete evaluation in addition to the best checkpoint performance reported. Diffusion Policy shows consistent

...


=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

| Lift | Pos | 2 | 8 | 10 | 9 | 22 | 8 | 256 | 0.3 | 1e-4 | 1e-3 | 100 | 100 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Can | Pos | 2 | 8 | 10 | 9 | 22 | 8 | 256 | 0.3 | 1e-4 | 1e-3 | 100 | 100 |
| Square | Pos | 2 | 8 | 10 | 9 | 22 | 8 | 256 | 0.3 | 1e-4 | 1e-3 | 100 | 100 |
| Transport | Pos | 2 | 8 | 10 | 9 | 45 | 8 | 256 | 0.3 | 1e-4 | 1e-3 | 100 | 100 |
| ToolHang | Pos | 2 | 8 | 10 | 9 | 22 | 8 | 256 | 0.3 | 1e-4 | 1e-3 | 100 | 100 |
| Push-T | Pos | 2 | 8 | 16 | 9 | 22 | 8 | 256 | 0.01 | 1e-4 | 1e-1 | 100 | 100 |
| Block Push | Vel | 3 | 1 | 5 | 9 | 0 | 8 | 256 | 0.3 | 1e-4 | 1e-3 | 100 | 100 |
| Kitchen | Pos | 4 | 8 | 16 | 80 | 0 | 8 | 768 | 0.1 | 1e-4 | 1e-3 | 10 

Rank 1 | Score -9.2736
TABLE VII: Hyperparameters for Transformer-based Diffusion Policy Ctrl: position or velocity control To: observ

Batches: 100%|██████████| 1/1 [00:00<00:00, 27.34it/s]



=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

| Lift | Pos | 2 | 8 | 16 | 2x84x84 | 2x76x76 | 256 | 22 | 1e-4 | 1e-6 | 100 | 100 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Can | Pos | 2 | 8 | 16 | 2x84x84 | 2x76x76 | 256 | 22 | 1e-4 | 1e-6 | 100 | 100 |
| Square | Pos | 2 | 8 | 16 | 2x84x84 | 2x76x76 | 256 | 22 | 1e-4 | 1e-6 | 100 | 100 |
| Transport | Pos | 2 | 8 | 16 | 4x84x85 | 4x76x76 | 264 | 45 | 1e-4 | 1e-6 | 100 | 100 |
| ToolHang | Pos | 2 | 8 | 16 | 2x240x240 | 2x216x216 | 256 | 22 | 1e-4 | 1e-6 | 100 | 100 |
| Push-T | Pos | 2 | 8 | 16 | 1x96x96 | 1x84x84 | 256 | 22 | 1e-4 | 1e-6 | 100 | 100 |
| Block Push | Pos | 3 | 1 | 12 | N/A | N/A | 256 | 0 | 1e-4 | 1e-6 | 100 | 100 |
| Kitchen | Pos | 2 | 8 | 16 | N/A | N/A 

Rank 1 | Score -9.0828
TABLE VI: Hyperparameters for CNN-based Diffusion Policy Ctrl: position or velocity control To: observation hor

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[Part 1/1] Ingested 67 chunks


In [28]:
retriever.search(["Describe the result TABLE II"])

[['Caption: TABLE IV: Multi-Stage Tasks (State Observation). For PushBlock,\npx is the frequency of pushing x blocks into the targets. For Kitchen,\npx is the frequency of interacting with x or more objects (e.g. bottom\nburner). Diffusion Policy performs better, especially for difficult\nmetrics such as p2 for Block Pushing and p4 for Kitchen, as\ndemonstrated by our results.\n Table: \n|  | BlockPush | BlockPush | Kitchen | Kitchen | Kitchen | Kitchen |\n| --- | --- | --- | --- | --- | --- | --- |\n|  | p1 | p2 | p1 | p2 | p3 | p4 |\n| LSTM-GMM [29] | 0.03 | 0.01 | 1.00 | 0.90 | 0.74 | 0.34 |\n| IBC [12] | 0.01 | 0.00 | 0.99 | 0.87 | 0.61 | 0.24 |\n| BET [42] | 0.96 | 0.71 | 0.99 | 0.93 | 0.71 | 0.44 |\n| DiffusionPolicy-C | 0.36 | 0.11 | 1.00 | 1.00 | 1.00 | 0.99 |\n| DiffusionPolicy-T | 0.99 | 0.94 | 1.00 | 0.99 | 0.99 | 0.96 |', 'Caption: TABLE II: Behavior Cloning Benchmark (Visual Policy) Performance are reported in the same format as in Tab I. LSTM-GMM numbers\nwere reproduced 

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.06it/s]


[[(np.float32(0.21874203),
   'Caption: TABLE II: Behavior Cloning Benchmark (Visual Policy) Performance are reported in the same format as in Tab I. LSTM-GMM numbers\nwere reproduced to get a complete evaluation in addition to the best checkpoint performance reported. Diffusion Policy shows consistent\nperformance improvement, especially for complex tasks like Transport and ToolHang.\n Table: \n| Task | # Rob | # Obj | ActD | #PH | #MH | Steps | Img? | HiPrec |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| Simulation Benchmark | Simulation Benchmark | Simulation Benchmark | Simulation Benchmark | Simulation Benchmark | Simulation Benchmark | Simulation Benchmark | Simulation Benchmark | Simulation Benchmark |\n| Lift | 1 | 1 | 7 | 200 | 300 | 400 | Yes | No |\n| Can | 1 | 1 | 7 | 200 | 300 | 400 | Yes | No |\n| Square | 1 | 1 | 7 | 200 | 300 | 400 | Yes | Yes |\n| Transport | 2 | 3 | 14 | 200 | 300 | 700 | Yes | No |\n| ToolHang | 1 | 2 | 7 | 200 | 0 | 700 | Yes | Yes |\

In [29]:
retriever.vector_store.collection.get()

{'ids': ['diffusion_policy_table_0',
  'diffusion_policy_table_1',
  'diffusion_policy_table_2',
  'diffusion_policy_table_3',
  'diffusion_policy_table_4',
  'diffusion_policy_table_5',
  'diffusion_policy_table_6',
  'diffusion_policy_table_7',
  'diffusion_policy_8',
  'diffusion_policy_9',
  'diffusion_policy_10',
  'diffusion_policy_11',
  'diffusion_policy_12',
  'diffusion_policy_13',
  'diffusion_policy_14',
  'diffusion_policy_15',
  'diffusion_policy_16',
  'diffusion_policy_17',
  'diffusion_policy_18',
  'diffusion_policy_19',
  'diffusion_policy_20',
  'diffusion_policy_21',
  'diffusion_policy_22',
  'diffusion_policy_23',
  'diffusion_policy_24',
  'diffusion_policy_25',
  'diffusion_policy_26',
  'diffusion_policy_27',
  'diffusion_policy_28',
  'diffusion_policy_29',
  'diffusion_policy_30',
  'diffusion_policy_31',
  'diffusion_policy_32',
  'diffusion_policy_33',
  'diffusion_policy_34',
  'diffusion_policy_35',
  'diffusion_policy_36',
  'diffusion_policy_37',
  'di

In [4]:
import shelve

db_file = 'data/models/user_data'
with shelve.open(db_file) as db:
    db["a"] = "abcd"